In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import json
from pathlib import Path
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Global constants (matching Milestone 1)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# Project paths
DATASET_PATH = "DATASET"
OUTPUT_DIR = "output"
MODEL_DIR = os.path.join(OUTPUT_DIR, "models")

# Create directories if they don't exist
for dir_path in [OUTPUT_DIR, MODEL_DIR]:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

print(" Environment setup complete")

In [ ]:
# Configure data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VALIDATION_SPLIT
)

# Create generators
print("Creating training generator...")
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

print("\nCreating validation generator...")
validation_generator = validation_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

NUM_CLASSES = len(train_generator.class_indices)
print(f"Number of classes: {NUM_CLASSES}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print("Class mapping:", train_generator.class_indices)

In [ ]:
# Create base model with pretrained weights
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze base model layers
base_model.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

In [ ]:
# Set up callbacks
checkpoint_path = os.path.join(MODEL_DIR, 'base_model_checkpoint.h5')
csv_path = os.path.join(OUTPUT_DIR, 'training_history.csv')

callbacks = [
    ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    CSVLogger(csv_path)
]

# Train the model
EPOCHS = 15
print("Training base model...")

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\n Base model training complete")

# Save the trained model
base_model_path = os.path.join(MODEL_DIR, 'base_model.h5')
model.save(base_model_path)
print(f"Model saved to {base_model_path}")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()

# Save the plot
history_plot_path = os.path.join(OUTPUT_DIR, 'training_history.png')
plt.savefig(history_plot_path)
plt.show()

print(f"\n Training history plot saved to {history_plot_path}")

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# Freeze first half of the base model layers
num_layers = len(base_model.layers)
freeze_until = num_layers // 2
for layer in base_model.layers[:freeze_until]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display updated model
model.summary()

# Set up callbacks for fine-tuning
fine_tune_checkpoint = os.path.join(MODEL_DIR, 'fine_tuned_checkpoint.h5')
fine_tune_csv = os.path.join(OUTPUT_DIR, 'fine_tuning_history.csv')

fine_tune_callbacks = [
    ModelCheckpoint(
        fine_tune_checkpoint,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    CSVLogger(fine_tune_csv)
]

# Fine-tune the model
print("\nFine-tuning the model...")
EPOCHS_FINE_TUNE = 10

fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=fine_tune_callbacks
)

# Save the fine-tuned model
fine_tuned_model_path = os.path.join(MODEL_DIR, 'fine_tuned_model.h5')
model.save(fine_tuned_model_path)
print(f"\n Fine-tuned model saved to {fine_tuned_model_path}")

In [ ]:
# Evaluate model on validation set
print("Evaluating final model...")
final_loss, final_accuracy = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // BATCH_SIZE
)

print(f"\nFinal Validation Loss: {final_loss:.4f}")
print(f"Final Validation Accuracy: {final_accuracy:.4f}")

# Generate predictions for confusion matrix
validation_generator.reset()
predictions = model.predict(
    validation_generator,
    steps=validation_generator.samples // BATCH_SIZE
)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = validation_generator.classes[:len(predicted_classes)]

# Generate classification report
class_names = list(validation_generator.class_indices.keys())
report = classification_report(
    true_classes,
    predicted_classes,
    target_names=class_names,
    output_dict=True
)

# Save classification report
report_path = os.path.join(OUTPUT_DIR, 'classification_report.json')
with open(report_path, 'w') as f:
    json.dump(report, f, indent=4)

# Create confusion matrix plot
plt.figure(figsize=(10, 8))
cm = confusion_matrix(true_classes, predicted_classes)
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    xticklabels=class_names,
    yticklabels=class_names
)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')

# Save confusion matrix plot
cm_path = os.path.join(OUTPUT_DIR, 'confusion_matrix.png')
plt.savefig(cm_path)
plt.show()

print(f"\n Evaluation metrics saved to {OUTPUT_DIR}")
print(f"Classification report: {report_path}")
print(f"Confusion matrix: {cm_path}")

In [ ]:
def predict_with_confidence(image_path, model, class_labels):
    """
    Predict skin condition with confidence scores
    """
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Get predictions
    predictions = model.predict(img_array)
    
    # Get confidence scores for each class
    results = {}
    for i, score in enumerate(predictions[0]):
        class_name = class_labels[i]
        confidence = float(score * 100)
        results[class_name] = confidence
        
    return results

def analyze_facial_aging(image_path, model, class_labels):
    """
    Analyze facial aging signs in an image and return predictions with confidence scores
    """
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Could not load image")
    
    # Convert to RGB for visualization
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Load the face cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Detect faces
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) == 0:
        # If no faces detected, analyze the whole image
        img_resized = cv2.resize(img, IMG_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img_resized)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        predictions = model.predict(img_array)
        
        # Create a result for the whole image
        results = [{
            'bbox': (0, 0, img.shape[1], img.shape[0]),
            'predictions': {
                class_labels[i]: float(score * 100)
                for i, score in enumerate(predictions[0])
            }
        }]
        return img_rgb, results
    
    results = []
    for (x, y, w, h) in faces:
        # Extract face ROI
        face_roi = img[y:y+h, x:x+w]
        
        # Preprocess ROI for model
        face_roi = cv2.resize(face_roi, IMG_SIZE)
        face_array = tf.keras.preprocessing.image.img_to_array(face_roi)
        face_array = np.expand_dims(face_array, axis=0)
        face_array = preprocess_input(face_array)
        
        # Get predictions
        predictions = model.predict(face_array)
        
        # Get confidence scores
        face_results = {
            'bbox': (x, y, w, h),
            'predictions': {}
        }
        
        for i, score in enumerate(predictions[0]):
            class_name = class_labels[i]
            confidence = float(score * 100)
            face_results['predictions'][class_name] = confidence
            
        results.append(face_results)
    
    return img_rgb, results

def visualize_predictions(image, results):
    """
    Visualize the predictions with bounding boxes and labels
    """
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    
    for result in results:
        x, y, w, h = result['bbox']
        predictions = result['predictions']
        
        # Draw bounding box
        rect = plt.Rectangle((x, y), w, h, fill=False, color='red', linewidth=2)
        plt.gca().add_patch(rect)
        
        # Format prediction text
        text = []
        for class_name, confidence in predictions.items():
            text.append(f"{class_name}: {confidence:.1f}%")
        
        # Add text above bounding box
        plt.text(x, y-10, '\n'.join(text), 
                bbox=dict(facecolor='white', alpha=0.7),
                fontsize=8, color='black')
    
    plt.axis('off')
    plt.title('Facial Aging Analysis')
    plt.tight_layout()
    return plt

In [ ]:
# Test image path
test_image_path = "test1.jpg"
print(f"\nAnalyzing {os.path.basename(test_image_path)}...")

try:
    # Analyze image
    image, results = analyze_facial_aging(test_image_path, model, class_names)
    
    # Visualize results
    plt = visualize_predictions(image, results)
    
    # Save the visualization
    output_path = os.path.join(OUTPUT_DIR, f"prediction_{os.path.basename(test_image_path)}")
    plt.savefig(output_path)
    plt.close()
    
    # Print detailed results
    print("\nDetailed Analysis:")
    for i, result in enumerate(results, 1):
        print(f"\nFace #{i}:")
        for class_name, confidence in result['predictions'].items():
            print(f"{class_name}: {confidence:.1f}%")
            
    print(f" Saved visualization to {output_path}")
        
except Exception as e:
    print(f"Error: {str(e)}")